In [110]:
import os
import json
import datetime
import pickle

from dotenv import load_dotenv

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from openai import OpenAI

from langchain import hub
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import MathpixPDFLoader, TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

## Settings

In [2]:
# Take environment variables from .env.
load_dotenv()

True

In [3]:
# OpenAI settings
openai_api_key: str = os.environ['OPENAI_API_KEY']
openai_api_version: str = '2023-05-15'
embedding_model: str = 'text-embedding-ada-002'

In [4]:
# Azure vector store settings
vector_store_address = os.environ['AZURE_VECTOR_STORE_ADDRESS']
vector_store_password = os.environ['AZURE_VECTOR_STORE_PASSWORD']

## Vector store

In [5]:
# OpenAIEmbeddings with OpenAI account
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_key = openai_api_key, 
    openai_api_version = openai_api_version, 
    model = embedding_model
)

In [6]:
# Create vector store
index_name: str = 'langchain-vector-demo'
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint = vector_store_address,
    azure_search_key = vector_store_password,
    index_name = index_name,
    embedding_function = embeddings.embed_query,
)

### Create embeddings

In [ ]:
# Embed text file
loader = TextLoader('/Users/jakob/Desktop/Lecture slides/Lorem.txt', encoding = 'utf-8')

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

In [ ]:
# Embed PDF file
loader = MathpixPDFLoader('/Users/jakob/Desktop/Lecture slides/l48-mlpw-06.pdf')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents = docs)

In [ ]:
# Embed CSV file
loader = CSVLoader('/Users/jakob/Desktop/q_hack/backend/lecture6.csv')
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(data)

vector_store.add_documents(documents = docs)

## Tools

### Calendar

In [7]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def authenticate():
    creds = None

    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        # If there are no (valid) credentials available, let the user log in.

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES
            )
            creds = flow.run_local_server(port = 0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

In [8]:
@tool
def fetch_exams(num_events = 10):
    '''
    Fetches the next num_events events from the user's Google Calendar

    Args:
        num_events (int): The number of events to fetch

    Returns:
        list: A list of the next num_events events
    '''

    upcoming_exams = []
    
    try:
        creds = authenticate()
        service = build('calendar', 'v3', credentials = creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        events_result = (
            service.events()
            .list(
                calendarId = 'primary',
                timeMin = now,
                maxResults = num_events,
                singleEvents = True,
                orderBy = 'startTime',
            )
            .execute()
        )
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return []

        # Prints the start and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            upcoming_exams.append((start, event['summary']))

    except HttpError as error:
        print(f'An error occurred: {error}')

    return upcoming_exams

### Fetch topics

In [9]:
# Hardcoded examples
courses = [
    'L46 - Principles of Machine Learning Systems',
    'L48 - ML and the Physical World',
    'R244 - Large-scale data processing and optimisation',
    'L45 - Representation Learning on Graphs and Networks',
    'R255 - Advanced Topics in Machine Learning'
]

lectures_per_course = {
    'L46 - Principles of Machine Learning Systems': [
        '1. The ML System Landscape',
        '2. Model Compression',
        '3. Mapping ML to Hardware',
        '4. Hardware Acceleration',
        '5. Training – Under the Hood',
        '6. Distributed Training',
        '7. Neural Architecture Search',
        '8. Federated Learning'
    ],
    'L48 - ML and the Physical World': [
        '1. Introduction',
        '2. Quantification of Beliefs',
        '3. Gaussian Processes',
        '4. Simulation',
        '5. Emulation',
        '6. Bayesian Optimisation',
        '7. Probabilistic Numerics',
        '8. Emukit and Experimental Design',
        '9. Sensitivity Analysis',
        '10. Multifidelity Modelling'
    ],
    'R244 - Large-scale data processing and optimisation': [
        '1. Introduction to Large-Scale Data Processing and Optimisation',
        '2. Data flow programming',
        '3. Large-scale graph data processing',
        '4. Data Flow Programming Tutorial',
        '5. Probabilistic Programming',
        '6. Optimisation in Computer Systems I',
        '7. Optimisation in Computer Systems II'
    ],
    'L45 - Representation Learning on Graphs and Networks': [
        '1. Introduction',
        '2. Complex Networks, Statistics on Nodes, Graph Laplacians, Network Model and Statistics, Node Embedding',
        '3. Permutation Invariance and Equivariance',
        '4. Graph Neural Networks',
        '5. Graph Rewiring',
        '6. Algorithmic Graph Nets',
        '7. Self-Supervised Graph Representation Learning',
        '8. Diffusion Models',
        '9. Neural Sheaf Diffusion for Deep Learning on Graphs',
        'Spacetime Dynamics and GNN',
        'Lifted and Long-Range Reasoning on Graphs',
        'On over-squashing in Graph Neural Networks: is the graph-topology a friend or an enemy?'
    ],
    'R255 - Advanced Topics in Machine Learning': [
        '1. Collective Intelligence',
        '2. Explainable AI',
        '3. Imitation Learning'
    ],
}

In [10]:
@tool
def fetch_topics_for_exam(exam: str):
    '''
    Fetches the topics for the given exam

    Args:
        exam (str): The name of the exam

    Returns:
        list: A list of topics for the given exam
    '''

    return lectures_per_course[exam]

### RAG

In [11]:
@tool
def retrieve_information(query: str):
    '''
    Retrieves information from a vector store.

    Args:
        query (str): The user query

    Returns:
        str: The retrieved information
    '''

    print('Retrieving information about: ', query)

    # Perform a similarity search
    docs = vector_store.similarity_search(
        query = query,
        k = 5,
        search_type = 'similarity',
    )

    return docs

In [39]:
client = OpenAI()

### Get YouTube snippet

In [168]:
@tool
def get_timestamp(query: str):
    '''
    Returns the start timestamp of a lecture recording based on the user query.

    Args:
        query (str): The user query

    Returns:
        str: The start timestamp of the lecture recording
    '''

    print('Getting timestamp for : ', query)

    # Perform a similarity search
    docs = vector_store.similarity_search(
        query = query,
        k = 5,
        search_type = 'similarity',
    )

    # Iterate over the documents and append all page_content to a string
    retrieved_chunks = ''
    for doc in docs:
        retrieved_chunks += doc.page_content + '\n'

    response = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        response_format = {'type': 'json_object'},
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant that returns the timestamp of a part of a lecture recording that the user is interested in. You output everything in a JSON format with a timestamp entry and lecture name entry.'},
            {'role': 'user', 'content': 'Please return the start timestamp for ' + query + ' based on the following information: ' + retrieved_chunks}
        ]
    )

    print(response.choices[0].message.content)

    parsed_response = json.loads(response.choices[0].message.content)

    # lecture_name = parsed_response['lecture_name']
    timestamp = parsed_response['timestamp']

    # Split the time string into hours, minutes, seconds, and microseconds
    hours, minutes, seconds_with_microseconds = timestamp.split(':')
    seconds, _ = seconds_with_microseconds.split('.')

    # Convert hours, minutes, seconds, and microseconds to integers
    hours = int(hours)
    minutes = int(minutes)
    seconds = int(seconds)

    # Calculate the total number of seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    return total_seconds

### Create flashcards

In [170]:
@tool
def create_flashcards(topic: str, num_flashcards: int = 3):
    '''
    Creates flashcards about a given topic.

    Args:
        topic (str): The topic to create flashcards for.
        num_flashcards (int): The number of flashcards to create.

    Returns:
        str: A message indicating that the flashcards have been created.
    '''

    # Perform a similarity search
    docs = vector_store.similarity_search(
        query = topic,
        k = 3,
        search_type = 'similarity',
    )

    # Iterate over the documents and append all page_content to a string
    retrieved_chunks = ''
    for doc in docs:
        retrieved_chunks += doc.page_content + '\n'

    response = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        response_format = {'type': 'json_object'},
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant that creates flashcards for students based on the provided information. You output everything in a JSON format as a list of questions and answers.'},
            {'role': 'user', 'content': 'Please create ' + str(num_flashcards) + ' flashcards based on the following information: ' + retrieved_chunks},
        ]
    )
    
    # Load the pickled flashcards file
    with open('flashcards.pkl', 'rb') as fp:
        flashcards = pickle.load(fp)

    # Append the new flashcards to the dictionary
    parsed_response = json.loads(response.choices[0].message.content)
    flashcards[topic] = parsed_response['flashcards']

    # Save the updated flashcards
    with open('flashcards.pkl', 'wb') as fp:
        pickle.dump(flashcards, fp)

    return 'The flashcards have been created.'

### Create quiz questions

In [252]:
@tool
def create_quiz_questions(topic: str, num_questions: int = 3):
    '''
    Creates quiz questions about a given topic.

    Args:
        topic (str): The topic to create quiz questions for.
        num_questions (int): The number of quiz questions to create.

    Returns:
        list: 

    '''

    # Perform a similarity search
    docs = vector_store.similarity_search(
        query = topic,
        k = 3,
        search_type = 'similarity',
    )

    # Iterate over the documents and append all page_content to a string
    retrieved_chunks = ''
    for doc in docs:
        retrieved_chunks += doc.page_content + '\n'

    response = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        response_format = {'type': 'json_object'},
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant that creates quizzes for students. You output everything in a JSON format where each entry is one quiz consisting of one question, four possible answers, and the index of the correct answer (0-based).'},
            {'role': 'user', 'content': 'Please create ' + str(num_questions) + ' quizzes. Base your questions and answers on the following knowledge pool: ' + retrieved_chunks},
        ]
    )

    return json.loads(response.choices[0].message.content)

## Agent (executor)

In [254]:
class StudentCopilot:
    def __init__(self):
        # Tools
        tools = [fetch_exams, fetch_topics_for_exam, retrieve_information, get_timestamp, create_flashcards, create_quiz_questions]

        # Stop tools
        self.stop_tools = [get_timestamp, create_quiz_questions]

        # Model
        model = ChatOpenAI(model = 'gpt-4-0125-preview', temperature = 0.0)

        # Prompt
        prompt = hub.pull('hwchase17/openai-tools-agent')

        # Create agent (executor)
        agent = create_openai_tools_agent(model, tools, prompt)
        self.agent_executor = AgentExecutor(agent = agent, tools = tools, return_intermediate_steps = True)

        # Chat history
        self.chat_history = []


    def run(self, input: str):
        # Invoke the agent
        result = self.agent_executor.invoke(
            {
                'input': input,
                'chat_history': self.chat_history,
            }
        )

        # Update chat history
        self.chat_history.append(HumanMessage(content = input))
        self.chat_history.append(AIMessage(content = result['output']))

        # Create default chat response
        response = {
            'return_type': 'chat',
            'text': result['output'],
        }

        # Parse intermediate steps
        for step in result['intermediate_steps']:
            action, tool_response = step
            print(step)
            # Timestamp
            if action.tool == 'get_timestamp':
                response = {
                    'return_type': 'timestamp',
                    'timestamp': tool_response,
                }
                break
            # Quiz questions
            elif action.tool == 'create_quiz_questions':
                response = {
                    'return_type': 'quiz_questions',
                    'quiz_questions': tool_response,
                }
                break

        return response

In [255]:
student_copilot = StudentCopilot()

In [257]:
greeting = 'How are you?'
upcoming_exams = 'What are my upcoming exams?'
topics = 'Yes, I want to prepare for my L45. What topics do I have to cover?'
bo_question = 'What does Bayesian Optimization look like in practice?'
flashcards = 'Create flashcards about acquisition functions.'
quiz = 'Create 3 quiz questions about bayesian optimization.'
timestamp = 'I want to rewatch the part about thompson sampling in the lecture on Bayesian Optimization. What was the timestamp?'

print(student_copilot.run(greeting))

{'return_type': 'chat', 'text': "I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?"}


## Flashcards

In [128]:
# Load flashcards
with open('flashcards.pkl', 'rb') as fp:
    flashcards = pickle.load(fp)

In [129]:
flashcards

{'acquisition functions': [{'question': 'What formula represents the acquisition function in Bayesian Optimization?',
   'answer': 'The acquisition function is represented as \\(\\alpha\\left(x ;\\left\\{x_{i}, y_{i}\\right\\}_{i=1}^{n}, \\mathcal{M}_{n}\\right) =\\mathbb{E}_{p(f)}[u(x)] =\\int u\\left(x, f\\left(x^{(*)}\\right), \\mathcal{M}_{n}\\right) p\\left(f \\mid\\left\\{x_{i}, y_{i}\\right\\}_{i=1}^{n}\\right) \\mathrm{d} f\\).'},
  {'question': 'What are the steps involved in Bayesian Optimization in practice?',
   'answer': 'The steps involved in Bayesian Optimization in practice are: 1) Encode your knowledge about the function in the GP prior. 2) Randomly sample some data. 3) Specify your acquisition function. 4) Evaluate and maximise the acquisition function. 5) Add new data to model and re-estimate hyperparameters. Loop steps 4 and 5 until the budget is gone.'},
  {'question': 'Why are Gaussian Processes (GPs) considered useful surrogates in Bayesian Optimization?',
   'an

In [130]:
# Reset flashcards
flashcards = {}
with open('flashcards.pkl', 'wb') as fp:
    pickle.dump(flashcards, fp)